# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31314    0 31314    0     0  89135      0 --:--:-- --:--:-- --:--:-- 88960


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70173    0 70173    0     0   598k      0 --:--:-- --:--:-- --:--:--  601k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [50]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [51]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

KnowledgeGraph(nodes: 14, relationships: 69)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 69)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

SingleHopSpecificQuerySynthesizer – Generates specific queries that only require one document to get obtain that answer
MultiHopAbstractQuerySynthesizer – This generates queries that require multiple steps of reasoning pulling information from multiple sources but requires reasoning and interpretation of the information to get the answer.
MultiHopSpecificQuerySynthesizer – This generates queries that require multiple steps of reasoning pulling information from multiple sources with very directly relevant information.

source: https://docs.ragas.io/en/stable/concepts/test_data_generation/rag/#query-types-in-rag


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Samples: 100%|██████████| 11/11 [00:57<00:00,  5.26s/it]


user_input  \
0   What is the significance of Mistral in the dev...   
1   What are the challenges associated with LLMs a...   
2   What LLMs do in 2023, they smart or dumb, how ...   
3   What was the most popular article about Bing i...   
4   What are the notable features of Google's Gemi...   
5   How has the environmental impact of AI models ...   
6   How have pricing changes in AI models influenc...   
7   How have pricing changes in AI models and prom...   
8   How did the advancements in Large Language Mod...   
9   How have advancements in GPT-4 and GPT-4o infl...   
10  How did the advancements in GPT-4 and other la...   

                                   reference_contexts  \
0   [Code may be the best application The ethics o...   
1   [Based Development As a computer scientist and...   
2   [Simon Willison’s Weblog Subscribe Stuff we fi...   
3   [easy to follow. The rest of the document incl...   
4   [Prompt driven app generation is a commodity a...   
5   [<1-hop>\n\nPrompt driven app generation is a ...   
6   [<1-hop>\n\nPrompt driven app generation is a ...   
7   [<1-hop>\n\nPrompt driven app generation is a ...   
8   [<1-hop>\n\nPrompt driven app generation is a ...   
9   [<1-hop>\n\nfeed with the model and talk about...   
10  [<1-hop>\n\nfeed with the model and talk about...   

                                            reference  \
0   Mistral is one of the organizations that have ...   
1   LLMs are described as convoluted black boxes, ...   
2   In 2023, Large Language Models (LLMs) were con...   
3   The most popular article about Bing in 2023 wa...   
4   Google's Gemini 1.5 Pro, released in February,...   
5   The environmental impact of AI models has seen...   
6   The pricing changes in AI models have signific...   
7   In 2024, the development and accessibility of ...   
8   In 2023 and 2024, significant advancements in ...   
9   Advancements in GPT-4 and GPT-4o have signific...   
10  In 2023, advancements in GPT-4 and other large...   

                        synthesizer_name  
0   single_hop_specifc_query_synthesizer  
1   single_hop_specifc_query_synthesizer  
2   single_hop_specifc_query_synthesizer  
3   single_hop_specifc_query_synthesizer  
4   single_hop_specifc_query_synthesizer  
5   multi_hop_abstract_query_synthesizer  
6   multi_hop_abstract_query_synthesizer  
7   multi_hop_abstract_query_synthesizer  
8   multi_hop_specific_query_synthesizer  
9   multi_hop_specific_query_synthesizer  
10  multi_hop_specific_query_synthesizer

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Generating Samples: 100%|██████████| 12/12 [01:17<00:00,  6.46s/it]


In [20]:
dataset.to_pandas()

user_input  \
0   What advancements has Mistral made in the fiel...   
1       Why Chinese grammar harder than code grammar?   
2   What were the significant advancements in Larg...   
3          Wht did Google say in the leaked document?   
4   How do the ethics of AI and the environmental ...   
5   How have advancements in Large Language Models...   
6   How do the challenges of understanding and con...   
7   How have advancements in Large Language Models...   
8   What are some of the challenges and unexpected...   
9   How has Meta's approach to training data and m...   
10  How has Meta contributed to the development an...   
11  How do GPT-4 and GPT-4o illustrate the advance...   

                                   reference_contexts  \
0   [Code may be the best application The ethics o...   
1   [Based Development As a computer scientist and...   
2   [Simon Willison’s Weblog Subscribe Stuff we fi...   
3   [easy to follow. The rest of the document incl...   
4   [<1-hop>\n\nCode may be the best application T...   
5   [<1-hop>\n\nCode may be the best application T...   
6   [<1-hop>\n\nCode may be the best application T...   
7   [<1-hop>\n\nCode may be the best application T...   
8   [<1-hop>\n\nSimon Willison’s Weblog Subscribe ...   
9   [<1-hop>\n\nAnother common technique is to use...   
10  [<1-hop>\n\nCode may be the best application T...   
11  [<1-hop>\n\nSimon Willison’s Weblog Subscribe ...   

                                            reference  \
0   The team behind Mistral is working to beat GPT...   
1   The grammar rules of programming languages lik...   
2   In 2023, Large Language Models (LLMs) were rec...   
3   The leaked Google document stated, "We Have No...   
4   The ethics of AI and the environmental impact ...   
5   Advancements in Large Language Models (LLMs) h...   
6   The challenges of understanding and controllin...   
7   Advancements in Large Language Models (LLMs) h...   
8   In 2024, one of the challenges with Large Lang...   
9   Meta's approach to training data, particularly...   
10  Meta has significantly contributed to the deve...   
11  GPT-4 and GPT-4o exemplify the advancements in...   

                        synthesizer_name  
0   single_hop_specifc_query_synthesizer  
1   single_hop_specifc_query_synthesizer  
2   single_hop_specifc_query_synthesizer  
3   single_hop_specifc_query_synthesizer  
4   multi_hop_abstract_query_synthesizer  
5   multi_hop_abstract_query_synthesizer  
6   multi_hop_abstract_query_synthesizer  
7   multi_hop_abstract_query_synthesizer  
8   multi_hop_specific_query_synthesizer  
9   multi_hop_specific_query_synthesizer  
10  multi_hop_specific_query_synthesizer  
11  multi_hop_specific_query_synthesizer

We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [25]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years! v2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [26]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [27]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [30]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [32]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [35]:
rag_chain.invoke({"question" : "What are Agents?"})

'The term "agents" refers to AI systems that can act on your behalf. However, it lacks a single, clear, and widely understood definition. Some view agents as things that perform tasks like a travel agent, while others think of them in terms of LLMs (Large Language Models) that use tools to solve problems. The concept of "autonomy" is often linked to agents, but without a clear definition. Overall, there is skepticism about their utility due to challenges like gullibility in making meaningful decisions.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [36]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [37]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: using built-in LangSmith QA evlaluator to look for correctness. grades respon as correct or incorrect based on reference answer.
- `labeled_helpfulness_evaluator`: custom evaluator that uses our prompt of helpfullness to evaluate the answer. providing reference labels
- `dope_or_nope_evaluator`: custom evaluator that uses our prompt of dope or nope to evaluate the answer.


source: https://docs.smith.langchain.com/old/evaluation/faq/evaluator-implementations


## LangSmith Evaluation

In [38]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'memorable-fuel-93' at:
https://smith.langchain.com/o/239bc38e-a472-4022-97f0-84cd498894d1/datasets/8345fffc-0927-46f2-aaed-79c7937ccfb4/compare?selectedSessions=ea3462b3-30d9-4749-ba99-66fd97e1ab36




12it [03:12, 16.03s/it]


<ExperimentResults memorable-fuel-93>

## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [39]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [40]:
rag_documents = docs

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Less larger chunks would likely lead to faster retrieval times as there are less vectors to compare and depending on number of chunks the duplicated overlap could be less. Depending on the context of the question, this could lead to lower accuracy as the context is more general or be better for accuracy if a larger context window is needed.

In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

A larger model improves search relevance and accuracy of embeddings but comes at higher compute cost and higher latency.

In [43]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [44]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [45]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [46]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents? Oh man, they’re like this elusive concept floating around in the AI universe. Some folks think of them as your personal assistants, ready to jump in and act on your behalf—like a digital travel agent. Others see them as LLMs (that’s Large Language Models, baby!) armed with tools to solve problems in a loop. But here\'s the kicker: nobody really agrees on what "agents" actually means. It’s like calling every cool gadget a "thing"—super vague! So while everyone’s hyped about them, they’re kinda like that dream vacation that keeps getting pushed back. Until they figure out this whole gullibility thing, the agent revolution might just be “coming soon.” Groovy, but a bit frustrating, right?'

Finally, we can evaluate the new chain on the same test set!

In [47]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'respectful-interest-59' at:
https://smith.langchain.com/o/239bc38e-a472-4022-97f0-84cd498894d1/datasets/8345fffc-0927-46f2-aaed-79c7937ccfb4/compare?selectedSessions=361bfa43-ba7f-4c6e-8f62-b76222681965




12it [03:47, 18.97s/it]


<ExperimentResults respectful-interest-59>

#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

The overall accuracy of the dope RAG chain is higher due to the modifications made.  
Utilizing a larger embedding model would lead to higher accuracy and improved scores across all 3 evaluators, with better details and relationships than the smaller model but would increase latency with higher inference times.

Utilizing a larger chunk size could reduce the token count as potentially needing less chunks with the same overlap count could lead to less overall token counts. It could reduce precision as well but would need to be tested with each dataset to narrow down the best ratios.

The dopeness specifically is higher because the prompt the second time around is more specifically asking for dope and cool responses. This will definitely give the responses a higher chance to be evaluated as dope.

Experiment comparison:
<img src="./screenshots/Screenshot from 2025-02-09 20-59-55.png" />

First run:
<img src="./screenshots/Screenshot from 2025-02-09 23-14-53.png" />

Second run:

<img src="./screenshots/Screenshot from 2025-02-09 23-14-59.png" />

